# Notebook 1: Breast Cancer Classification using Gene Expression Data

## Learning Objectives
- Download and explore RNAseq data from the Cancer Genome Atlas in a SageMaker Studio Notebook.
- Train a classifier using the open-source XGBoost library.

## Environment Notes
This notebook was last tested on a `ml.t3.medium (2 vCPU + 4 GiB)` instance running the `Python 3 (Data Science)` kernel in SageMaker Studio.

## Table of Contents
1. [Background](#1.-Background)
2. [Data Preparation](#2.-Data-Preparation)
    1. [Download and Unzip Data](#2.A.-Download-and-Unzip-Data)
    2. [Load Normalized Gene Expression Data](#2.B.-Load-Normalized-Gene-Expression-Data)
    3. [Load Phenotype Data](#2.C.-Load-Phenotype-Data)
    4. [Merge the Normalized Gene Expression and Phenotype Data](#2.D.-Merge-the-Normalized-Gene-Expression-and-Phenotype-Data)
    5. [Encode Target Variable](#2.E.-Encode-Target-Variable)
    6. [Split the Data into Training-Validation-Test Sets](#2.F.-Split-the-Data-into-Training-Validation-Test-Sets)
3. [Model Training](#3.-Model-Training)
    1. [Train XGBoost Model](#3.A.-Train-XGBoost-Model)
    2. [Evaluate the Model Performance](#3.B.-Evaluate-the-Model-Performance)

## 1. Background

The increase in biological sequence data and development of sophisticated analysis techniques promises to dramatically improve healthcare. The "Central Dogma" of biology claims that the human body operates by transcribing DNA into messenger RNA, which is then translated into proteins. Each step in this process is managed by sequence information. So, the hope of bioinformatics is to decode these sequences and provide insight into their effect and possible alteration.

![alt text](img/640px-Gene_structure_eukaryote_2_annotated.png "Central Dogma")

*The structure and expression of a eukaryotic protein-coding gene. WikiJournal of Medicine 4 (1). DOI:10.15347/wjm/2017.002. ISSN 20024436. [(Wikimedia Commons)](https://commons.wikimedia.org/wiki/File:Gene_structure_eukaryote_2_annotated.svg)*

One of the more recent advances in sequence analysis is RNA sequencing (RNAseq). By using high-throughput sequencers, scientists can now capture a "snapshot" of what mRNA is present in a sample, and thus what genes are expressed, at any given time. RNAseq does not require prior knowledge of what genes to target and can capture alternatively-spliced variants which have been shown to have a significant impact on disease.

In this notebook, we'll demonstrate using RNAseq from the [Cancer Genome Atlas](https://www.nature.com/articles/nature11412) data to predict HER2 status, a binary classification task. Breast cancers with HER2-overexpression/amplification [(HER2+)](https://www.cancer.org/cancer/breast-cancer/understanding-a-breast-cancer-diagnosis/breast-cancer-her2-status.html) have a more aggressive natural history and are responsive to HER2-directed therapies.

## 2. Data Preparation

### 2.A. Download and Unzip Data

In this step, we download and extract two datasets from [The Cancer Genome Atlas](https://www.cancer.gov/about-nci/organization/ccg/research/structural-genomics/tcga):

* [Normalized Gene Expression Data (1,218 samples)](https://xenabrowser.net/datapages/?dataset=TCGA.BRCA.sampleMap%2FHiSeqV2_PANCAN&host=https%3A%2F%2Ftcga.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443): TCGA breast invasive carcinoma (BRCA) gene expression data measured experimentally using the IlluminaHiSeq RNAseq platform, mean-normalized (per gene) across all TCGA cohorts. Values in this dataset are generated at UCSC by combining gene expression RNAseq values of all TCGA cohorts. Values for each gene are then mean-centered per gene the data split back into cohorts.

* [Phenotypes (1,247 samples)](https://xenabrowser.net/datapages/?dataset=TCGA.BRCA.sampleMap%2FBRCA_clinicalMatrix&host=https%3A%2F%2Ftcga.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443): TCGA breast invasive carcinoma (BRCA) clinically-observed phenotypes.

In [ ]:
%pip install -r requirements.txt -q -q

In [ ]:
import argparse
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, f1_score
import xgboost as xgb

In [ ]:
# Define working directories
WORKING_DIR = os.getcwd()
DATA_DIR = os.path.join(WORKING_DIR, "data")
print(f"Working directory is {WORKING_DIR}")
print(f"Data directory is {DATA_DIR}")

# Get TCGA BRCA Gene Expression Data
!wget https://tcga.xenahubs.net/download/TCGA.BRCA.sampleMap/HiSeqV2_PANCAN.gz -nc -P $DATA_DIR/input/raw/
!gzip -df $DATA_DIR/input/raw/HiSeqV2_PANCAN.gz

# Get TCGA BRCA Phenotype Data
!wget https://tcga.xenahubs.net/download/TCGA.BRCA.sampleMap/BRCA_clinicalMatrix -nc -P $DATA_DIR/input/raw/

### 2.B. Load Normalized Gene Expression Data

In [ ]:
# Read in RNAseq data file
genom = pd.read_csv(f"{DATA_DIR}/input/raw/HiSeqV2_PANCAN", sep='\t')
print(f"The gene expression data has {genom.shape[0]} records and {genom.shape[1]} columns\n")

# Extract list of sample identifiers
genom_identifiers = genom["sample"].values.tolist()

# Print the first few records
print("Example gene expression data:")
genom.head(3)

### 2.C. Load Phenotype Data

In [ ]:
phenotypes = pd.read_csv(f"{DATA_DIR}/input/raw/BRCA_clinicalMatrix", sep='\t')
print(f"The phenotype data has {phenotypes.shape[0]} records and {phenotypes.shape[1]} columns.\n")

# Print the first few records
print("Example phenotype data:")
phenotypes.loc[[0,100,200,300,400,500]]

There are a lot of potential phenotypic values to choose as targets. For this example, let's focus our attention on `HER2_Final_Status_nature2012`.

Let's replace any missing HER2 values with `Negative`

In [ ]:
phenotypes_subset = phenotypes[["sampleID", "HER2_Final_Status_nature2012"]].reset_index(drop=True)
print(f"Of the {phenotypes_subset.shape[0]} records, only {phenotypes_subset.dropna().shape[0]} have complete HER2 values. Let's replace the missing values with 'Negative'.\n")

phenotypes_subset.fillna("Negative", inplace=True)
print("Example phenotype data:")
phenotypes_subset.head(10)

### 2.D. Merge the Normalized Gene Expression and Phenotype Data

The gene expression and phenotype data are oriented in different directions, so let's transpose the former before we merge them together.

In [ ]:
genom_transpose = genom.set_index("sample").transpose().reset_index().rename(columns={"index": "sampleID"})
print(f"Transposing the gene expression data changes its shape from {genom.shape[0]} x {genom.shape[1]} to {genom_transpose.shape[0]} x {genom_transpose.shape[1]}.\n")

# Merge the phenotype data with the transposed gene expression data on `sampleID`
df = pd.merge(phenotypes_subset, genom_transpose, on="sampleID", how="outer")
print(f"The merged data set has {df.shape[0]} records and {df.shape[1]} columns.\n")

# Let's look at the results for 6 specific samples.
print("Example transposed gene expression data:")
df.loc[[0,100,200,300,400,500]]

### 2.E. Encode Target Variable

In [ ]:
df["target"] = [0 if t == "Negative" else 1 for t in df['HER2_Final_Status_nature2012']]
df.insert(loc=1, column='target', value=df.pop('target'))

# Drop any rows with NaN values
df = df.dropna()

df.loc[[0,100,200,300,400,500]]

What's the distribution between our positive and negative classes?

In [ ]:
print("How many of the samples are positive for HER2?")
print(df.target.value_counts())

Since only 10% of the samples are positive, we may want to weight our classes differently during training

Here are the `target` values for our six samples of interest.

In [ ]:
df.loc[[0,100,200,300,400,500],['sampleID','HER2_Final_Status_nature2012','target']]

Let's remove the original target value and sampleID to prevent data leakage

In [ ]:
df = df.drop(['HER2_Final_Status_nature2012','sampleID'], axis=1)

Finally, let's review the transformed data for our six samples of interest.

In [ ]:
df.loc[[0,100,200,300,400,500]]

### 2.F. Split the Data into Training-Validation-Test Sets

In [ ]:
# Hold out 20% of the data for testing
train_df, test_df = train_test_split(df, test_size=0.2)

# Hold out an additional 20% of the training data for validaton
train_df, val_df = train_test_split(train_df, test_size=0.2)

print(f"The training data has {train_df.shape[0]} records and {train_df.shape[1]} columns.")
print(f"The validation data has {val_df.shape[0]} records and {val_df.shape[1]} columns.")
print(f"The test data has {test_df.shape[0]} records and {test_df.shape[1]} columns.")

Convert labels and features into numpy arrays

In [ ]:
train_labels = np.array(train_df.pop("target"))
val_labels = np.array(val_df.pop("target"))
test_labels = np.array(test_df.pop("target"))

train_np = np.array(train_df)
val_np = np.array(val_df)
test_np = np.array(test_df)

## 3. Model Training

### 3.A. Train XGBoost Model

In [ ]:
hyper_params_dict = {
    'objective':             "binary:logistic",
    'booster':               "gbtree",
    'eval_metric':           "error",
    'n_estimators':          15,
    'max_depth':             3, 
    'min_child_weight':      5,
    'subsample':             0.9,
    'gamma':                 0.01
}
    
# Use the scale_pos_weight parameter to account for the imbalanced classes in our data
pos_weight = float(np.sum(train_labels == 0) / np.sum(train_labels == 1))
    
breast_cancer_classifier = xgb.XGBClassifier(
    **hyper_params_dict, 
    use_label_encoder=False, 
    scale_pos_weight=pos_weight # Use pos_weight value calculated above to account for unbalanced classes
)

breast_cancer_classifier.fit(
    train_np,
    train_labels,
    eval_set=[(train_np, train_labels), (val_np, val_labels)], 
    verbose=True
)

### 3.B. Evaluate the Model Performance

Plot learning curves

In [ ]:
results = breast_cancer_classifier.evals_result()
plt.plot(results['validation_0']['error'], label='train')
plt.plot(results['validation_1']['error'], label='val')
plt.legend()
plt.show()

Evaluate model accuracy on test data

In [ ]:
# Create a custom function for generating a confusion matrix for a given p-value
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    
    if len(set(list(labels))) == 2:
        print('Correctly un-detected (True Negatives): ', cm[0][0])
        print('Incorrectly detected (False Positives): ', cm[0][1])
        print('Misses (False Negatives): ', cm[1][0])
        print('Hits (True Positives): ', cm[1][1])
        print('Total: ', np.sum(cm[1]))

In [ ]:
# evaluate predictions
test_predictions = breast_cancer_classifier.predict(test_np)
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

plot_cm(test_labels, np.array(test_predictions))

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1:.2f}")